In [ ]:
from tabulate import tabulate
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

In [ ]:
# ganti nama file dari hasil tahap normalisasi sebelumnya
file = pd.read_csv('data_OUT_TOTAL.csv', delimiter=',')

In [ ]:
len(file)

In [ ]:
# ganti nama file_input dengan file .txt file asli dari hasil preprocessing
file_input = "data_TOTAL.txt"
file_txt = open(file_input, mode='r', encoding='utf-8')
text = file_txt.readlines()

In [ ]:
len(text)

In [ ]:
file['tweet_id'].mode()[0]

In [ ]:
# Code untuk mencari fitur POS
list_feature = []

for idx, t in enumerate(text):
    selected = file[file['tweet_id'] == idx]
    list_tweet = []
    
    if len(selected.index) > 0:
        for a, tweet in selected.iterrows():
            feature = float(tweet['normalize_weight'])
            list_tweet.append(feature)
            
            
        if len(selected.index) < pd.value_counts(file['tweet_id']).max():
            b = len(selected.index)
            while b < pd.value_counts(file['tweet_id']).max():
                feature = float(0)
                b += 1
                list_tweet.append(feature)

        
    elif len(selected.index) == 0:
        i = 0
        while i < pd.value_counts(file['tweet_id']).max():
            feature = float(0)
            list_tweet.append(feature)
            i += 1
            
    list_feature.append(list_tweet)
    
data = np.array(list_feature)
#data = sparse.csr_matrix(sX)

# format dimenasi yang didapat menggunakan fitur POS
print(data)

In [ ]:
# mencari label tweetnya dari file .txt hasil preprocessing
lt = []

for i,t in enumerate(text):
    a = t.split(',')
    label_y = a[1]
    label_y = label_y.split('\n')
    lt.append(label_y[0])

label_target = np.array(lt)
# label_target = sparse.csr_matrix(sY.astype(float))

In [ ]:
print(label_target)

In [ ]:
# code untuk mencari TERM PRESENCE
tp = []
df = file.drop_duplicates(subset=['term'])

print(len(df))
print(pd.value_counts(file['tweet_id']).max())

for j in text:
    tt = []
    tmp = j.split(',')
    tmp = tmp[0].split()
    for k in range(len(df['term'])):
        dd = df.iloc[k]['term']
        #print(dd)
        if any(dd in s for s in tmp):
            tt.append(float(1))
        else:
            tt.append(float(0))
    
    tp.append(tt)

term_presence = np.array(tp)

print(len(term_presence))
print(term_presence)

In [ ]:
# code untuk menggabungkan fitur TERM PRESENCE dan POS Tagging
data_total = np.column_stack([data, term_presence])

print(data_total.shape)
print(data_total)

In [ ]:
# fungsi untuk menampilkan confussion matrix pada hasil klasifikasi
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    #if normalize:
    #    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    #    print("Normalized confusion matrix")
    #else:
    #    print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
%%time
# code untuk Intelligent Model Selection
SVCclassification_1 = svm.SVC(kernel='rbf')
SVCclassification_2 = svm.SVC(kernel='rbf')

# tunning nilai K untuk membagi jumlah dataset menjadi traning, testing, dan evaluasi
skf = KFold(n_splits=10)
target_names = ['Positif','Negatif']

itter = 1
ltt = []
dtt = []
best_model_tt = []
label_best_model_tt = []
evdtt = []
evltt = []
s = []
r = []
p = []
f1 = []

for train_test_index, validation_index in skf.split(data_total, label_target):
    
    ltt.clear()
    dtt.clear()
    evdtt.clear()
    evltt.clear()
    
    # mencari label
    #print(validation_index)
    for i in train_test_index:
        ltt.append(label_target[i])
        dtt.append(data_total[i])
        
    label_testing_test = np.array(ltt)
    data_testing_test = np.array(dtt)
    
    # find best model in second cross validation with 90% data
    score2 = cross_val_score(SVCclassification_1, data_testing_test, label_testing_test, cv=skf)
    
    #print all score from second CV
    #print("this is score : ", score2)
    
    # find index best score from second cross validation
    best_score_index = np.argmax(score2)
    
    #print best index model
    #print("this is best model index : ",best_score_index)
    
    counter = 0
    for f_index,g_index in skf.split(data_testing_test,label_testing_test):
        if counter == best_score_index:
            best_model_tt.clear()
            label_best_model_tt.clear()
            for c in f_index:
                best_model_tt.append(data_testing_test[c])
                label_best_model_tt.append(label_testing_test[c])       
            best_model = np.array(best_model_tt)    
            best_model_label = np.array(label_best_model_tt)
            break
        counter += 1 
    
    # save evaluation test
    for j in validation_index:
        evltt.append(label_target[j])
        evdtt.append(data_total[j])
    
    evaluation_datatest = np.array(evdtt)
    evaluation_label_datatest = np.array(evltt)
    
    # fit BEST model with evaluation
    SVCclassification_2.fit(best_model, best_model_label)    
    
    #print final accuracy (done with evalution set)
    print("Best Final Accuracy on %0.0f CV: %0.3f" % (itter, SVCclassification_2.score(evaluation_datatest, evaluation_label_datatest)))
    itter += 1
    
    y_predic = SVCclassification_2.predict(evaluation_datatest)
    #print(y_predic)
    #print(evaluation_label_datatest)
    
    print(classification_report(evaluation_label_datatest, y_predic, target_names=target_names))
    #print(confusion_matrix(evaluation_label_datatest, y_predic))
    
    # menghitung confusion matrix
    cnf_matrix = confusion_matrix(evaluation_label_datatest, y_predic)
    np.set_printoptions(precision=2)

    # menampilkan Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=target_names,title='Confusion matrix, without normalization')
    plt.show()
    
    # menyimpan hasil score , precision, recall, f1-score pada setiap iterasi
    s.append(SVCclassification_2.score(evaluation_datatest, evaluation_label_datatest))
    p.append(precision_score(evaluation_label_datatest, y_predic, pos_label='1'))
    r.append(recall_score(evaluation_label_datatest, y_predic, pos_label='1'))
    f1.append(f1_score(evaluation_label_datatest, y_predic, pos_label='1'))
    
save_accuracy_np = np.array(s)
#print("acc : ",save_accuracy_np)

save_recall_np = np.array(r)
#print("recall : " ,save_recall_np)

save_precision_np = np.array(p)
#print("precision : ", save_precision_np)

save_f1_np = np.array(f1)
#print("f1 : " , save_f1_np)

In [ ]:
print("Accuracy: %0.3f (+/- %0.3f)" % (np.average(save_accuracy_np), np.std(save_accuracy_np)*2)) 
print("Precision: %0.3f" % (np.average(save_precision_np))) 
print("Recall: %0.3f" % (np.average(save_recall_np))) 
print("f1 score: %0.3f" % (np.average(save_f1_np)))

In [ ]:
# cross validation normal tanpa IMS
# target_names = ['negatif', 'positif']
# clf = svm.SVC(kernel='linear')
# skf = KFold(n_splits=6)
# scores = cross_val_score(clf, data_total, label_target, cv=skf)
# predicted = cross_val_predict(clf, data_total, label_target, cv=skf)

# print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std()*2))

# print(classification_report(label_target, predicted, target_names=target_names))

In [ ]:
# plt.figure(figsize=(12, 6))
# sns.barplot(x=["IMS", "normal CV"], y=[np.average(save_accuracy_np), scores.mean()])